#### Aqui iniciei com o import do selenium
###### Mas antes foi preciso fazer a instalação das bibliotecas:
- pip install selenium
- pip install scrapy
- pip install ipython
- pip install pandas

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import re
import datetime
import pandas as pd

##### Fiz a instalação do chromedriver e em seguida abro a página do Google

In [2]:

# Configurar o driver do navegador (exemplo usando o Chrome)
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.google.com")

##### Em seguida, insiro a palavra chave para busca, e abro o Linkedin

In [3]:
driver.find_element('name', 'q').send_keys('vagas marketing')
driver.get("https://www.linkedin.com")

###### Ao abrir o Linkedin, não irei logar, portanto seleciono button para vagas

In [4]:
sleep(3)
driver.find_elements(by=By.XPATH, value='/html/body/nav/ul/li[4]/a')[0].click()

###### Em seguida, limpo a label de localidade e a label de palavras-chaves para vagas, e insiro a que quero buscar

In [5]:
driver.find_element('name','keywords').clear()
driver.find_element('name','keywords').send_keys('Marketing E Publicidade')
driver.find_element('name','location').clear()
driver.find_element('name','location').send_keys('Brasil')

###### Seleciono o localidade 1 das opções que aparece ao preencher a label

In [6]:
sleep(3)
driver.find_elements(by=By.XPATH, value='//*[@id="location-1"]')[0].click()

###### Aqui irei abrir os filtros para tipos de vaga, e selecionar o filtro Tempo integral e Estágio, por último seleciono o botão concluído

In [11]:
sleep(3)
driver.find_elements(by=By.XPATH, value='//*[@id="jserp-filters"]/ul/li[4]/div/div/button')[0].click()

In [12]:
sleep(3)
driver.find_elements(by=By.XPATH, value='//*[@id="jserp-filters"]/ul/li[4]/div/div/div/div/div/div[1]/label')[0].click()

In [13]:
sleep(3)
driver.find_elements(by=By.XPATH, value='//*[@id="jserp-filters"]/ul/li[4]/div/div/div/button')[0].click()

In [14]:
sleep(3)
# fecho evento de iniciar sessão
try:
    evento_sessao = driver.find_elements(by=By.XPATH, value='/html/body/div[3]/button')[0]
    # Se o elemento estiver presente, clica nele para fechar o evento
    evento_sessao.click()
except:
    # Se o elemento não estiver presente, não faz nada
    pass

##### Realizando Scrolling na página até o limite

In [15]:
'''# Realizar o scrolling até o fim da página
contador = 0
altura_anterior = driver.execute_script("return window.pageYOffset;")
while contador < 2:
    # Realizar o scrolling até o fim da página
    actions = ActionChains(driver)
    actions.send_keys(Keys.END).perform()
    sleep(2)  # Aguardar um tempo para carregar mais vagas
    
    # Verificar se não há mais vagas para carregar (chegou ao fim da página)
    try:
        even_ver_mais = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[2]/button')[0]
        #even_ver_mais.click()
        driver.execute_script("arguments[0].click();", even_ver_mais)
        # Verificar se a altura da página foi alterada
        altura_atual = driver.execute_script("return window.pageYOffset;")
        if altura_atual == altura_anterior:
            # Incrementar o contador quando o botão não estiver mais disponível
            contador += 1
        else:
            # Zerar o contador quando a página for expandida
            contador = 0
        altura_anterior = altura_atual
    except NoSuchElementException:
         # Incrementar o contador quando o botão não estiver mais disponível
        contador += 1
        continue
'''

'# Realizar o scrolling até o fim da página\ncontador = 0\naltura_anterior = driver.execute_script("return window.pageYOffset;")\nwhile contador < 2:\n    # Realizar o scrolling até o fim da página\n    actions = ActionChains(driver)\n    actions.send_keys(Keys.END).perform()\n    sleep(2)  # Aguardar um tempo para carregar mais vagas\n    \n    # Verificar se não há mais vagas para carregar (chegou ao fim da página)\n    try:\n        even_ver_mais = driver.find_elements(by=By.XPATH, value=\'//*[@id="main-content"]/section[2]/button\')[0]\n        #even_ver_mais.click()\n        driver.execute_script("arguments[0].click();", even_ver_mais)\n        # Verificar se a altura da página foi alterada\n        altura_atual = driver.execute_script("return window.pageYOffset;")\n        if altura_atual == altura_anterior:\n            # Incrementar o contador quando o botão não estiver mais disponível\n            contador += 1\n        else:\n            # Zerar o contador quando a página for

###### Pegando link das vagas

In [16]:
vagas_links = driver.find_elements(By.XPATH, '/html/body/div[1]/div/main/section[2]/ul/li/div/a')
vagas_links = [link.get_attribute('href') for link in vagas_links]

###### Pegando titulos das vagas

In [17]:
vagas_title = driver.find_elements(by=By.CLASS_NAME, value='base-search-card__title')
vagas_title=[titulo.text for titulo in vagas_title]


###### Pegando nome das empresas das vagas

In [18]:
vagas_empresa = driver.find_elements(by=By.CLASS_NAME, value='hidden-nested-link')
vagas_empresa = [empresa.text for empresa in vagas_empresa]


###### Pegando link das empresas das vagas e os dados de tipo e nivel de experiência

In [19]:
print(len(vagas_links))
print(len(vagas_title))
print(len(vagas_empresa))

25
25
25


In [20]:
def rasp(tipo, nivel, candidaturas, URL):
    vagas_nivel = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[1]/span')
    for i in vagas_nivel:
        nivel.append(i.text)
    
    vagas_tipo = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[2]/span')
    for i in vagas_tipo:
        tipo.append(i.text)
    
    try:
        vagas_candidaturas = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h4/div[2]/figure/figcaption')
        if vagas_candidaturas[0].text == "Seja um dos 25 primeiros a se candidatar":
            candidaturas.append("Menos de 25 candidaturas")
        else:
            candidaturas.append(vagas_candidaturas[0].text)
    except:
        vagas_candidaturas = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h4/div[2]/span[2]')
        candidaturas.append(vagas_candidaturas[0].text)
        
    vagas_URL = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/div/button[1]')
    for i in vagas_URL:
        URL.append(i.get_attribute('href'))
   

In [21]:
def rasp_info_emp(seguidores_emp, local_emp, qtd_funcionarios):
    #tratar erro de quando não encontrar informações de texto sobre seguidores no linkedin
    try:
        elements = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/section/div/div[2]/div[1]/h3')
        if len(elements) > 0:
            info_aux = re.split(padrao, elements[0].text, maxsplit = 1)
            seguidores_emp.append(info_aux[1]+info_aux[2])
        else:
            seguidores_emp.append("Não informado")

    except NoSuchElementException:
        seguidores_emp.append("Não informado")


    element_loc = driver.find_elements(by=By.XPATH, value='//*[@id="address-0"]')
    local_emp.append(element_loc[0].text)
    

    element_funcionario = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/section[1]/div/dl/div[3]/dd')
    qtd_funcionarios.append(element_funcionario[0].text)

In [23]:
vagas_tipo = []
vagas_nivel = []
num_candidaturas = []
URL_candidatura = []
links_sites_emp = []
horario_Scraping = []

seguidores_empresa, local_empresa, qtd_func = [], [], []
#para separar a string quando encontrar o primeiro digito
padrao = r"(\d)" #significa um digito

j = 0
for i in vagas_links:
    agora = datetime.datetime.now()
    formatado = agora.strftime("%d/%m/%Y %H:%M")
    #print(formatado)
    horario_Scraping.append(formatado)
    sleep(2)
    driver.get(i)
    
    #verificar se o url aberto foi o certo
    while driver.current_url != i:
        driver.get(i)
        #print(j,driver.current_url)
    
    rasp(vagas_nivel, vagas_tipo, num_candidaturas, URL_candidatura)
    
    site_empresa = driver.find_elements(by=By.XPATH, value='//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[1]/a')
    for k in site_empresa:
        links_sites_emp.append(k.get_attribute('href'))
    
    #Abrir site da empresa
    driver.get(links_sites_emp[j])

    #verificar se o url aberto foi o certo
    while driver.current_url != links_sites_emp[j]:
        driver.get(links_sites_emp[j])

    sleep(3)

    # fecho evento de logar se estiver presente
    try:
        evento_entrar = driver.find_elements(by=By.XPATH, value='//*[@id="organization_guest_contextual-sign-in"]/div/section/button')[0]
        # Se o elemento estiver presente, clica nele para fechar o evento
        evento_entrar.click()
    except:
        # Se o elemento não estiver presente, não faz nada
        pass
    
    rasp_info_emp(seguidores_empresa, local_empresa, qtd_func)
    
    j = j + 1
    

In [24]:
print(vagas_links)
print(vagas_title)
print(vagas_empresa)
print(vagas_tipo)
print(vagas_nivel)

print(num_candidaturas)
print(URL_candidatura)

print(links_sites_emp)
print(horario_Scraping)

print(seguidores_empresa)
print(local_empresa)

print(qtd_func)

['https://br.linkedin.com/jobs/view/analista-de-comunica%C3%A7%C3%A3o-e-marketing-junior-at-sicoob-agrocredi-3557022573?refId=H14PBnuDDD%2BiA5EH9jJzTQ%3D%3D&trackingId=wd%2BUTOHvVpntGDcIrMUulA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://br.linkedin.com/jobs/view/brand-designer-at-nelogica-3531121540?refId=H14PBnuDDD%2BiA5EH9jJzTQ%3D%3D&trackingId=pZfiWyBxEHAiuWNKLmxHWw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://br.linkedin.com/jobs/view/superior-em-publicidade-e-propaganda-ou-marketing-at-instituto-formar-3555791539?refId=H14PBnuDDD%2BiA5EH9jJzTQ%3D%3D&trackingId=bJ8l258xB4Khe4%2Fw4OX9%2FA%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://br.linkedin.com/jobs/view/assistente-de-marketing-e-publicidade-at-equipe-cine-3521828190?refId=H14PBnuDDD%2BiA5EH9jJzTQ%3D%3D&trackingId=Clk9JHoTuH0ii8eCKO83Pg%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card', 'https://br.linkedin.c

In [25]:
col = ["URL da vaga no linkedin", "Nome da vaga", 
                 "Nome da empresa contratante","URL da empresa contratante",
                "Tipo de contratação", "Nível de experiência", "Número de candidaturas para vaga",
                 "Horário do scraping", "Número de funcionários da empresa", "Número de seguidores da empresa",
                "Local sede da empresa"]
df = pd.DataFrame({"URL da vaga no linkedin":vagas_links, "Nome da vaga":vagas_title, 
                 "Nome da empresa contratante":vagas_empresa,"URL da empresa contratante":links_sites_emp,
                "Tipo de contratação":vagas_tipo, "Nível de experiência":vagas_nivel, "Número de candidaturas para vaga":num_candidaturas,
                   "Horário do scraping": horario_Scraping, "Número de funcionários da empresa":qtd_func,
                   "Número de seguidores da empresa":seguidores_empresa,"Local sede da empresa":local_empresa})



In [26]:
df.head()

,URL da vaga no linkedin,Nome da vaga,Nome da empresa contratante,URL da empresa contratante,Tipo de contratação,Nível de experiência,Número de candidaturas para vaga,Horário do scraping,Número de funcionários da empresa,Número de seguidores da empresa,Local sede da empresa
0,https://br.linkedin.com/jobs/view/analista-de-...,Analista de Comunicação e Marketing Junior,Sicoob Agrocredi,https://br.linkedin.com/company/sicoob-agrocre...,Não aplicável,Tempo integral,Menos de 25 candidaturas,05/04/2023 13:18,201-500 funcionários,4.660 seguidores,"R Capitao Joaquim Norberto, 105\nGuaxupe, Mg 3..."
1,https://br.linkedin.com/jobs/view/brand-design...,Brand Designer,Nelogica,https://br.linkedin.com/company/nelogica?trk=p...,Assistente,Tempo integral,Mais de 200 candidaturas,05/04/2023 13:18,501-1.000 funcionários,302.851 seguidores,"Avenida Carlos Gomes 400\nPorto Alegre, Rio Gr..."
2,https://br.linkedin.com/jobs/view/superior-em-...,SUPERIOR EM PUBLICIDADE E PROPAGANDA ou MARKETING,Instituto Formar,https://br.linkedin.com/company/instituto-form...,Assistente,Tempo integral,Menos de 25 candidaturas,05/04/2023 13:18,11-50 funcionários,512 seguidores,"R Pedro Zangrande 662\nSerra, Espírito Santo 2..."
3,https://br.linkedin.com/jobs/view/assistente-d...,Assistente de Marketing e Publicidade,Equipe Cine,https://br.linkedin.com/company/equipe-cine?tr...,Assistente,Tempo integral,71 candidaturas,05/04/2023 13:18,51-200 funcionários,Não informado,"Rua Gaspar Soares 226\nSão Paulo, BR"
4,https://br.linkedin.com/jobs/view/assistente-d...,Assistente de Mídias Sociais/Digitais ( Social...,Fundação São Paulo,https://br.linkedin.com/company/funda%C3%A7%C3...,Não aplicável,Tempo integral,Mais de 200 candidaturas,05/04/2023 13:19,1.001-5.000 funcionários,2.932 seguidores,"Rua João Ramalho\n182\nSão Paulo, SP 05008000, BR"


In [27]:
df.to_csv("Scraping - Francisco Davi Gomes de Oliveira.csv", mode='w')